# Bash Tool with E2B Sandbox

Demonstration of the bash tool functionality using E2B sandbox to execute shell commands in a remote environment.

In [1]:
// Import E2B SDK and load environment
import { Sandbox } from "https://esm.sh/e2b@latest";
import * as dotenv from "jsr:@std/dotenv";

// Import the bash tool
import { bashTool, handleBash } from '../src/tools/bash.ts';
import { StreamData } from 'npm:ai';

// Load environment variables
await dotenv.load({ export: true });

const apiKey = Deno.env.get("E2B_API_KEY");
if (!apiKey) {
  throw new Error("E2B_API_KEY environment variable is required");
}

console.log("Environment loaded successfully");

Environment loaded successfully


In [2]:
// Create sandbox and clone repository
const sandbox = await Sandbox.create({
    template: "base",
    apiKey: apiKey,
    timeoutMs: 600_000 // 10 minutes
  });
  
  console.log(`Sandbox created with ID: ${sandbox.id}`);
  
  // Clone the Sudoku Solver repository
  const cloneResult = await sandbox.commands.run("git clone https://github.com/Rahimeen-Altaf/Sudoku-Solver-DSA.git /home/user/sudoku-solver");
  console.log("Repository cloned successfully");
  
  console.log("Sandbox ready for testing");

  // Set up mock objects for testing
const mockData = new StreamData();
const mockMessages: any[] = [];
const mockAppendToolCallMessage = (msg: any) => [msg];

console.log('Tool definition:', bashTool);

Sandbox created with ID: undefined
Repository cloned successfully
Sandbox ready for testing
Tool definition: {
  type: "function",
  function: {
    name: "bash",
    description: "Execute bash commands in the sandbox environment.",
    parameters: {
      type: "object",
      properties: {
        command: { type: "string", description: "The bash command to execute" },
        working_directory: {
          type: "string",
          description: "Optional working directory to run the command in"
        },
        timeout: {
          type: "number",
          description: "Optional timeout in milliseconds (default: 30000)"
        }
      },
      required: [ "command" ]
    }
  }
}


In [3]:
// Test basic commands - list directory contents
const lsToolCall = {
  id: 'test-ls',
  func: {
    name: 'bash',
    arguments: {
      command: 'ls -la /home/user/sudoku-solver'
    }
  }
};

console.log('=== Testing Basic Command - ls ===');
const lsResult = await handleBash(
  lsToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Command result:', lsResult[0].tool_call_result);
console.log('Directory listing:');
console.log(lsResult[0].tool_call_result.stdout);

=== Testing Basic Command - ls ===
Command result: {
  success: true,
  stdout: "total 40\n" +
    "drwxr-xr-x 4 user user 4096 Sep 22 19:49 .\n" +
    "drwx------ 3 user user 4096 Sep 22 19:49 ..\n" +
    "drwxr-xr-x 8 user user 4096 Sep 22 19:49 .git\n" +
    "-rw-r--r-- 1 user user 2356 Sep 22 19:49 README.md\n" +
    "drwxr-xr-x 2 user user 4096 Sep 22 19:49 __pycache__\n" +
    "-rw-r--r-- 1 user user 1098 Sep 22 19:49 puzzles.py\n" +
    "-rw-r--r-- 1 user user  833 Sep 22 19:49 puzzles.txt\n" +
    "-rw-r--r-- 1 user user 1134 Sep 22 19:49 solutions.py\n" +
    "-rw-r--r-- 1 user user 5952 Sep 22 19:49 solver.py\n",
  stderr: "",
  exit_code: 0
}
Directory listing:
total 40
drwxr-xr-x 4 user user 4096 Sep 22 19:49 .
drwx------ 3 user user 4096 Sep 22 19:49 ..
drwxr-xr-x 8 user user 4096 Sep 22 19:49 .git
-rw-r--r-- 1 user user 2356 Sep 22 19:49 README.md
drwxr-xr-x 2 user user 4096 Sep 22 19:49 __pycache__
-rw-r--r-- 1 user user 1098 Sep 22 19:49 puzzles.py
-rw-r--r-- 1 user use

In [ ]:
// Clean up sandbox
await sandbox.kill();
console.log('Sandbox closed successfully');